In [7]:
pip install pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 340.0 kB/s eta 0:00:0000:0100:02
Note: you may need to restart the kernel to use updated packages.


In [47]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Given Data
plants = ['USA', 'Germany', 'Brazil', 'India', 'Japan']
markets = ['North America', 'South America', 'Europe', 'Japan', 'Asia']

# Capacities (tons)
capacity = {'USA': 185, 'Germany': 475, 'Brazil': 200, 'India': 80, 'Japan': 50}

# Minimum operation constraint (50% capacity if open)
min_capacity = {p: 0.5 * capacity[p] for p in plants}

# Demand at each market (tons)
demand = {'North America': 270, 'South America': 190, 'Europe': 200, 'Japan': 120, 'Asia': 100}

# Production cost per ton (in local currency)
production_cost = {'USA': 10000, 'Germany': 15000, 'Brazil': 13000, 'India': 400000, 'Japan': 1800000}

# Exchange rates (convert to USD)
exchange_rate = {'USA': 1, 'Germany': 0.502, 'Brazil': 0.562, 'India': 0.023, 'Japan': 0.0093}

# Transportation costs (USD per ton)
transport_cost = {
    'USA': [600, 1200, 1300, 2000, 1700],
    'Germany': [1300, 1400, 600, 1400, 1300],
    'Brazil': [1200, 800, 1400, 2100, 2100],
    'India': [2200, 2300, 1300, 1000, 800],
    'Japan': [2000, 2100, 1400, 300, 900]
}
transport_cost = {p: {markets[i]: transport_cost[p][i] for i in range(len(markets))} for p in plants}

# Decision variables
x = { (p, m): LpVariable(f"x_{p}_{m}", lowBound=0) for p in plants for m in markets }
y = { p: LpVariable(f"y_{p}", cat="Binary") for p in plants }

# Define Problem
prob = LpProblem("Sunchem_Production_Plan", LpMinimize)

# Objective Function: Minimize total cost
prob += lpSum(
    (production_cost[p] * exchange_rate[p] * lpSum(x[p, m] for m in markets)) + 
    (lpSum(transport_cost[p][m] * x[p, m] for m in markets)) 
    for p in plants
)

# Demand Constraints: Each market demand must be met
for m in markets:
    prob += lpSum(x[p, m] for p in plants) == demand[m]

# Capacity Constraints: Cannot exceed plant capacity
for p in plants:
    prob += lpSum(x[p, m] for m in markets) <= capacity[p] * y[p]

# Minimum Operation Constraint: If a plant operates, it must be at least 50% capacity
for p in plants:
    prob += lpSum(x[p, m] for m in markets) >= min_capacity[p] * y[p]

# Solve Problem
prob.solve()

# Display Results
print("Optimal Production Plan:")
for p in plants:
    if y[p].varValue == 1:
        print(f"\nPlant {p} is Open:")
        for m in markets:
            if x[p, m].varValue > 0:
                print(f"  - Ship {x[p, m].varValue} tons to {m}")

print("\nTotal Cost:", value(prob.objective))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/dk/ty44k4f97tl78kr03dymv92c0000gn/T/a655231bb9d44207ace311e07fd13cf7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/dk/ty44k4f97tl78kr03dymv92c0000gn/T/a655231bb9d44207ace311e07fd13cf7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 141 RHS
At line 157 BOUNDS
At line 163 ENDATA
Problem MODEL has 15 rows, 30 columns and 85 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7.81645e+06 - 0.00 seconds
Cgl0004I processed model has 15 rows, 30 columns (5 integer (5 of which binary)) and 85 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.324324
Cbc0038I Solution found of 7.81645e+06
Cbc0038I Relaxing continuous gives 7.81645e+06
Cbc0038I Before mi

In [49]:
capacity['Germany'] += 10  # Adding 10 tons to Germany plant
prob.solve()
print("\nUpdated Total Cost after Adding 10 Tons Capacity:", value(prob.objective))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/dk/ty44k4f97tl78kr03dymv92c0000gn/T/edbed534571d4b2d950ef958151197f9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/dk/ty44k4f97tl78kr03dymv92c0000gn/T/edbed534571d4b2d950ef958151197f9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 141 RHS
At line 157 BOUNDS
At line 163 ENDATA
Problem MODEL has 15 rows, 30 columns and 85 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7.81645e+06 - 0.00 seconds
Cgl0004I processed model has 15 rows, 30 columns (5 integer (5 of which binary)) and 85 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.324324
Cbc0038I Solution found of 7.81645e+06
Cbc0038I Relaxing continuous gives 7.81645e+06
Cbc0038I Before mi